# Hello World

In [15]:
from crewai_tools import ScrapeWebsiteTool, FileWriterTool, TXTSearchTool
import requests
from crewai import Agent, Task, Crew, Process
import os
from crewai_tools import TXTSearchTool

In [16]:
# Initialize the tool, potentially passing the session
scrape_tool = ScrapeWebsiteTool(website_url="https://en.wikipedia.org/wiki/Artificial_intelligence")

# Extract the text 
text = scrape_tool.run()
print(text[:100])  # Print the first 500 characters of the scraped text for verification

Using Tool: Read website content
The following text is scraped website content:
Artificial intelligence - Wikipedia
Jump to content
M


In [17]:
# Initialize the file writer tool
file_writer = FileWriterTool()

# Write content to a file in a specified directory
result = file_writer.run(filename="ai.txt", content=text, directory="data", overwrite=True)
print(result)  # Print the result of the file writing operation

Using Tool: File Writer Tool
Content successfully written to data/ai.txt


In [18]:
import dotenv

dotenv.load_dotenv()  # Load environment variables from a .env file if present

openai_api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = openai_api_key

In [19]:
# Initialize the tool with a specific text file, so the agent can search within the given file's content
# use chromadb to chunk and vectorize data
search_tool = TXTSearchTool(txt="data/ai.txt")

In [20]:
data_analyst = Agent(
    role='Educator',
    goal=f'Based on the context provided, answer the question.',
    backstory='You are a data expert',
    verbose=True,
    allow_delegation=False,
    tools=[search_tool]
)

test_task = Task(
    description="Understand the topic of Natural Language Processing and summarize it for me",
    agent=data_analyst,
    expected_output="I want the response to be as short as possible."
)

crew = Crew(
    tasks=[test_task],
    process=Process.sequential
)

output = crew.kickoff()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Educator                                                                                                │
│                                                                                                                 │
│  Task: Understand the topic of Natural Language Processing and summarize it for me                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Relevant Content:                                                                                              │
│                                                                                                                 │
│  Natural language processing (NLP) allows programs to read, write and communicate in human languages. [ 50 ]    │
│  Specific problems include speech recognition , speech synthesis , machine translation , information            │
│  extraction , information retrieval and question answering . [ 51 ]                                             │
│                                                                                                                 │
│  Early work, based on Noam Chomsky 's generative grammar and semantic networks , had difficulty with            │
│  word-sense disambiguation [ f ] unless restricted to small domains called " micro-worlds " (due to the common  │
│  sense knowledge problem [ 29 ] ). Margaret Masterman believed that it was meaning and not grammar that was     │
│  the key to understanding languages, and that thesauri and not dictionaries should be the basis of              │
│  computational language structure.                                                                              │
│                                                                                                                 │
│   Modern deep learning techniques for NLP include word embedding (representing words, typically as vectors      │
│  encoding their meaning), [ 52 ] transformers (a deep learning architecture using an attention mechanism), [    │
│  53 ] and others. [ 54 ] In 2019, generative pre-trained transformer (or "GPT") language models began to        │
│  generate coherent text, [ 55 ] [ 56 ] and by 2023, these models were able to get human-level scores on the     │
│  bar exam , SAT test, GRE test, and many other real-world applications. [ 57 ]                                  │
│                                                                                                                 │
│  Perception                                                                                                     │
│                                                                                                                 │
│  Machine perception is the ability to use input from sensors (such as cameras, microphones, wireless signals,   │
│  active lidar , sonar, radar, and tactile sensors ) to deduce aspects of the world. Computer vision is the      │
│  ability to analyze visual input. [ 58 ]                                                                        │
│                                                                                                                 │
│                                                                                                                 │
│  Natural language processing (NLP) allows programs to read, write and communicate in human languages. [ 50 ]    │
│  Specific problems include speech recognition , speech synthesis , machine translation , information            │
│  extraction , information retrieval and question answering . [ 51 ]                                             │
│                                                                                                                 │
│  Early work, based on Noam Chomsky 's generative grammar and semantic networks , had difficulty with            │
│  word-sense disambiguation [ f ] unless restricted to small domains called " micro-worlds " (due to the ...     │
│                                                                                                                 │
╰───────────────────────────────────────────────────────

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Educator                                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Natural language processing (NLP) allows programs to read, write and communicate in human languages. Specific  │
│  problems include speech recognition, speech synthesis, machine translation, information extraction,            │
│  information retrieval and question answering. Early work, based on Noam Chomsky's generative grammar and       │
│  semantic networks, had difficulty with word-sense disambiguation unless restricted to small domains called     │
│  "micro-worlds" (due to the common sense knowledge problem). Margaret Masterman believed that it was meaning    │
│  and not grammar that was the key to understanding languages, and that thesauri and not dictionaries should be  │
│  the basis of computational language structure. Modern deep learning techniques for NLP include word embedding  │
│  (representing words, typically as vectors encoding their meaning), transformers (a deep learning architecture  │
│  using an attention mechanism), and others. In 2019, generative pre-trained transformer (or "GPT") language     │
│  models began to generate coherent text, and by 2023, these models were able to get human-level scores on the   │
│  bar exam, SAT test, GRE test, and many other real-world applications.                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [23]:
from langchain_openai import ChatOpenAI

nlp_task = Task(
    description="Understand the topic of Natural Language Processing and give the correct response",
    # agent=data_analyst, # commenting out the agent now, letting the manager decide
    expected_output="I want the response to be as short as possible."
)

calculator_agent = Agent(
    role='Calculator',
    goal=f'You calculate things',
    backstory='You love Maths',
    verbose=True,
    allow_delegation=False,
    tools=[]
)

math_task = Task(
    description="Tell me what 123 * 45 is",
    expected_output="Calculate this"
)

crew = Crew(
    tasks=[nlp_task, math_task],
    agents=[data_analyst, calculator_agent],
    process=Process.hierarchical,
    manager_llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7),
    verbose=True
)

crew.kickoff()

╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  e017050a-87e0-4479-81bb-fcf148292b6e                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name: Understand the topic of Natural Language Processing and give the correct response                        │
│  ID: ad8aa16f-ce09-409c-bea5-93604dabcd47                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│  Task: Understand the topic of Natural Language Processing and give the correct response                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── 🔧 Tool Execution Started (#1) ─────────────────────────────────────────╮
│                                                                                                                 │
│  Tool: Delegate work to coworker                                                                                │
│  Args: {"task": "Understand the topic of Natural Language Processing and provide a short response", "context":  │
│  "The response should be concise and to the point, focusing on the key aspects of Natural Languag...            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Educator                                                                                                │
│                                                                                                                 │
│  Task: Understand the topic of Natural Language Processing and provide a short response                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  123 * 45 = 5535                                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│  Tell me what 123 * 45 is                                                                                       │
│  Agent:                                                                                                         │
│  Crew Manager                                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

CrewOutput(raw='123 * 45 = 5535', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Understand the topic of Natural Language Processing and give the correct response', name='Understand the topic of Natural Language Processing and give the correct response', expected_output='I want the response to be as short as possible.', summary='Understand the topic of Natural Language Processing and give the...', raw='Natural language processing (NLP) allows programs to read, write, and communicate in human languages. Specific applications include speech recognition, machine translation, information extraction, and question answering. Early approaches focused on generative grammar and semantic networks, facing challenges with word-sense disambiguation. Contemporary NLP leverages techniques like word embedding and transformers to achieve advanced language understanding and generation capabilities. In recent years, models like GPT have achieved human-level performance on various la

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  e017050a-87e0-4479-81bb-fcf148292b6e                                                                           │
│  Final Output: 123 * 45 = 5535                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯